**ENTRENAMIENTO**

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.io as io
import numpy as np
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.utils.class_weight import compute_class_weight
from efficientnet_pytorch import EfficientNet
import glob
from torchvision.io import read_image
import gc
import kornia.augmentation as K

# ===============================
# Configuración
# ===============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Usando {device}")

transform_gpu = nn.Sequential(
    K.RandomHorizontalFlip(p=0.5),
    K.RandomRotation(degrees=30.0),
    K.RandomBrightness(0.2),
    K.Normalize(mean=torch.tensor([0.5]*3), std=torch.tensor([0.5]*3))
).to(device)


NUM_CLASSES = 14
BATCH_SIZE = 8
EPOCHS_FROZEN = 30
EPOCHS_UNFROZEN = 25
LR_FROZEN = 1e-3
LR_UNFROZEN = 1e-5
IMG_SIZE = 384


def convertir_csv_a_pt(csv_path, output_dir, batch_size=5000):
    os.makedirs(output_dir, exist_ok=True)
    df = pd.read_csv(csv_path)
    images, labels = [], []

    for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Procesando {os.path.basename(csv_path)}"):
        ruta = row["ruta"]
        clase = int(row["clase"])
        preprocess = transforms.Compose([
            transforms.Resize((IMG_SIZE, IMG_SIZE)),
            transforms.ConvertImageDtype(torch.float32),
            transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
        ])

        img = read_image(ruta)
        if img.shape[0] == 1:
            img = img.repeat(3, 1, 1)
        img = preprocess(img)
        images.append(img)
        labels.append(clase)

        if len(images) == batch_size or idx == len(df) - 1:
            batch_id = idx // batch_size
            torch.save({
                "images": torch.stack(images),
                "labels": torch.tensor(labels, dtype=torch.long)
            }, os.path.join(output_dir, f"batch_{batch_id}.pt"))
            print(f"✅ Guardado {output_dir}/batch_{batch_id}.pt")
            images, labels = [], []
            torch.cuda.empty_cache()
            gc.collect()

def verificar_o_convertir(csv_path, output_dir):
    archivos = glob.glob(os.path.join(output_dir, "*.pt"))
    if len(archivos) > 0:
        print(f" {output_dir} ya tiene {len(archivos)} archivos .pt. No se convierte de nuevo.")
    else:
        print(f" No hay archivos .pt en {output_dir}. Se crearán a partir del CSV.")
        convertir_csv_a_pt(csv_path, output_dir)


# ===============================
# Transformaciones
# ===============================
def get_transforms(train=True):
    if train:
        return A.Compose([
            A.Rotate(limit=30),
            A.HorizontalFlip(),
            A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15),
            A.RandomBrightnessContrast(),
            A.Normalize(mean=(0.5,), std=(0.5,)),
            ToTensorV2(),
        ])
    else:
        return A.Compose([
            A.Normalize(mean=(0.5,), std=(0.5,)),
            ToTensorV2(),
        ])


# ===============================
# Dataset desde CSV
# ===============================
class PTBatchDataset(torch.utils.data.Dataset):
    def __init__(self, pt_file, transform=None):
        data = torch.load(pt_file)
        self.images = data["images"]
        self.labels = data["labels"]
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        x = self.images[idx].to(device)
        y = self.labels[idx].to(device)

        if self.transform:
            x = self.transform(x.unsqueeze(0)).squeeze(0)

        return x, y




# Funciones para checkpoint
def save_checkpoint(model, optimizer, epoch, path="checkpoint_efficientnetUCD.pth"):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, path)
    print(f" Checkpoint guardado en epoch {epoch}")

def load_checkpoint(model, optimizer, path="checkpoint_efficientnetUCD.pth"):
    if os.path.exists(path):
        checkpoint = torch.load(path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print(f" Checkpoint cargado desde epoch {checkpoint['epoch'] + 1}")
        return checkpoint['epoch'] + 1
    print(" No se encontró checkpoint. Empezando desde 0.")
    return 0


# ===============================
# Datos y loaders
# ===============================
train_csv = "D:/Mi unidad/TESIS/MURA_UCD/train_multiclase_corr.csv"
valid_csv = "D:/Mi unidad/TESIS/MURA_UCD/valid_multiclase_corr.csv"

verificar_o_convertir(train_csv, "dataset_pt/train_UCD")
verificar_o_convertir(valid_csv, "dataset_pt/valid_UCD")

train_pt_files = sorted(glob.glob("dataset_pt/train_UCD/*.pt"))
valid_pt_files = sorted(glob.glob("dataset_pt/valid_UCD/*.pt"))




# ===============================
#  Pesos de clase
# ===============================
y_train = pd.read_csv(train_csv)["clase"].values
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y_train), y=y_train)
weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)
print("✅ Pesos de clase:", weights_tensor)

# ===============================
#  Modelo EfficientNetB4
# ===============================
model = EfficientNet.from_pretrained("efficientnet-b4", num_classes=NUM_CLASSES)
model = model.to(device)

# Congelar todas menos últimas capas
for param in model.parameters():
    param.requires_grad = False
for param in model._fc.parameters():
    param.requires_grad = True

# ===============================
# Fase 1: capas congeladas
# ===============================
# Inicialización para fase 1
optimizer = torch.optim.Adam(model.parameters(), lr=LR_FROZEN)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.2)
criterion = nn.CrossEntropyLoss(weight=weights_tensor)
scaler = torch.cuda.amp.GradScaler()

start_epoch = load_checkpoint(model, optimizer)

# ===============================
# Entrenamiento por archivo .pt
# ===============================
print("Iniciando entrenamiento")

for epoch in range(start_epoch, EPOCHS_FROZEN + EPOCHS_UNFROZEN):

    if epoch == EPOCHS_FROZEN:
        print("Fase 2: Fine-tuning completo")
        for param in model.parameters():
            param.requires_grad = True
        optimizer = torch.optim.Adam(model.parameters(), lr=LR_UNFROZEN)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.2)

    model.train()
    correct, total, loss_acum = 0, 0, 0
    fase = "F1" if epoch < EPOCHS_FROZEN else "F2"

    for pt_file in train_pt_files:
        dataset = PTBatchDataset(pt_file, transform=transform_gpu)


        train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

        loop = tqdm(train_loader, desc=f"{fase} Epoch {epoch+1} - {os.path.basename(pt_file)}")

        for images, labels in loop:
            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            loss_acum += loss.item()
            loop.set_postfix(loss=loss_acum / (total / BATCH_SIZE), acc=correct / total)

        #  Liberar memoria por archivo
        del dataset, train_loader, images, labels, outputs, preds
        torch.cuda.empty_cache()
        gc.collect()

    scheduler.step(loss_acum / (total / BATCH_SIZE))
    save_checkpoint(model, optimizer, epoch)


# ===============================
# Guardar modelo final
# ===============================
torch.save(model.state_dict(), "efficientnet_multiclaseUCD.pth")
print("✅ Modelo final guardado como efficientnet_multiclaseUCD.pth")


✅ Usando cuda
 dataset_pt/train_UCD ya tiene 5 archivos .pt. No se convierte de nuevo.
 dataset_pt/valid_UCD ya tiene 2 archivos .pt. No se convierte de nuevo.
✅ Pesos de clase: tensor([0.6366, 0.6456, 0.8933, 1.2893, 0.8371, 1.2857, 2.2023, 3.4428, 0.6729,
        1.6971, 3.5387, 3.8812, 0.4675, 0.6607], device='cuda:0')
Loaded pretrained weights for efficientnet-b4


C:\Users\andre\AppData\Local\Temp\ipykernel_2020\1050083405.py:190: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
C:\Users\andre\AppData\Local\Temp\ipykernel_2020\1050083405.py:139: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `w

 Checkpoint cargado desde epoch 50
Iniciando entrenamiento


C:\Users\andre\AppData\Local\Temp\ipykernel_2020\1050083405.py:108: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(pt_file)
F2 Epoch 51 - batch_0.pt:   0%| 

 Checkpoint guardado en epoch 50


...

 Checkpoint guardado en epoch 52


F2 Epoch 54 - batch_4.pt: 100%|███████████████████████████████| 625/625 [01:09<00:00,  9.03it/s, acc=0.881, loss=0.258]


 Checkpoint guardado en epoch 53


F2 Epoch 55 - batch_4.pt: 100%|█████████████████████████████████| 625/625 [01:09<00:00,  8.98it/s, acc=0.88, loss=0.26]


 Checkpoint guardado en epoch 54
✅ Modelo final guardado como efficientnet_multiclaseUCD.pth


**PREDICCIONES**

**CURVA DE ROC POR CLASES Y AUC**

In [ ]:
from sklearn.metrics import classification_report
from torch.utils.data import TensorDataset

all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    for pt_file in valid_pt_files:  # <--- CORREGIDO
        data = torch.load(pt_file)
        X = data["images"]
        y = data["labels"]
        val_loader = DataLoader(TensorDataset(X, y), batch_size=BATCH_SIZE, shuffle=False)

        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

        del data, X, y, val_loader
        torch.cuda.empty_cache()
        gc.collect()

# ===============================
# Reporte de clasificación
# ===============================
target_names = [f"Clase {i}" for i in range(NUM_CLASSES)]
reporte = classification_report(all_labels, all_preds, target_names=target_names, digits=3)
print("📊 Reporte de Clasificación:\n")
print(reporte)


C:\Users\andre\AppData\Local\Temp\ipykernel_2020\1965974550.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(pt_file)


📊 Reporte de Clasificación:

              precision    recall  f1-score   support

     Clase 0      0.734     0.867     0.795       905
     Clase 1      0.834     0.687     0.753       905
     Clase 2      0.810     0.892     0.849       601
     Clase 3      0.843     0.728     0.781       463
     Clase 4      0.755     0.902     0.822       621
     Clase 5      0.824     0.588     0.686       437
     Clase 6      0.812     0.924     0.864       276
     Clase 7      0.805     0.642     0.714       148
     Clase 8      0.848     0.840     0.844       933
     Clase 9      0.572     0.607     0.589       326
    Clase 10      0.791     0.863     0.825       175
    Clase 11      0.857     0.680     0.758       150
    Clase 12      0.830     0.915     0.870      1239
    Clase 13      0.866     0.726     0.790       822

    accuracy                          0.802      8001
   macro avg      0.798     0.776     0.782      8001
weighted avg      0.806     0.802     0.799      80

In [ ]:
# Convertir etiquetas a one-hot
y_true = label_binarize(all_labels, classes=list(range(NUM_CLASSES)))
y_score = all_probs  # ya es (N, 14)

fpr = dict()
tpr = dict()
roc_auc = dict()

os.makedirs("roc_por_clase", exist_ok=True)

for i in range(NUM_CLASSES):
    fpr[i], tpr[i], _ = roc_curve(y_true[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

    # Plot
    plt.figure(figsize=(6, 5))
    plt.plot(fpr[i], tpr[i], color='blue', lw=2, label=f"AUC = {roc_auc[i]:.2f}")
    plt.plot([0, 1], [0, 1], 'k--', lw=1)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("Falsos Positivos")
    plt.ylabel("Verdaderos Positivos")
    plt.title(f"Curva ROC - Clase {i}")
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"roc_por_clase/roc_clase_{i}.png")
    plt.close()
    print(f"✅ Curva ROC guardada para Clase {i}")


✅ Curva ROC guardada para Clase 0
✅ Curva ROC guardada para Clase 1
✅ Curva ROC guardada para Clase 2
✅ Curva ROC guardada para Clase 3
✅ Curva ROC guardada para Clase 4
✅ Curva ROC guardada para Clase 5
✅ Curva ROC guardada para Clase 6
✅ Curva ROC guardada para Clase 7
✅ Curva ROC guardada para Clase 8
✅ Curva ROC guardada para Clase 9
✅ Curva ROC guardada para Clase 10
✅ Curva ROC guardada para Clase 11
✅ Curva ROC guardada para Clase 12
✅ Curva ROC guardada para Clase 13


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt

# ===============================
# Predicciones finales por clase
# ===============================
y_true = all_labels
y_pred = np.argmax(all_probs, axis=1)

# ===============================
# Matriz de confusión
# ===============================
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=[f"C{i}" for i in range(NUM_CLASSES)],
            yticklabels=[f"C{i}" for i in range(NUM_CLASSES)])
plt.xlabel("Predicción")
plt.ylabel("Valor real")
plt.title("Matriz de Confusión - Validación")
plt.tight_layout()
plt.savefig("matriz_confusion.png")
plt.close()
print("✅ Matriz de confusión guardada como matriz_confusion.png")


✅ Matriz de confusión guardada como matriz_confusion.png
